In [1]:
#Cria sessão Spark
from pyspark.sql import SparkSession
from pyspark.sql.functions import col
import pyspark.sql.functions as F

In [2]:
#Instância Spark
spark = SparkSession.builder.appName('Curso Pyspark') \
         .config('spark.sql.repl.eagerEval.enabled', True) \
         .getOrCreate()      

In [3]:
# lendo um arquivo parquet
df = spark.read.parquet('./DATASETS/LOGINS.parquet')

In [4]:
df.show(20, truncate=0)

+--------------+----------------------------------+----------+------------------+------+-------------+---------------+------------+---------------------------+-------------------+
|cpf           |email                             |senha     |data_de_nascimento|estado|data_cadastro|ipv4           |cor_favorita|profissao                  |telefone           |
+--------------+----------------------------------+----------+------------------+------+-------------+---------------+------------+---------------------------+-------------------+
|981.507.362-12|pedro-lucas53@gmail.com           |+7^7E%xFBc|2006-12-18        |RR    |2023-02-26   |99.107.250.210 |Roxo        |Jogador De Golfe           |31 7785-4046       |
|493.705.168-75|rezendeisaac@hotmail.com          |_O_2GRnGOe|1992-06-17        |GO    |2023-02-16   |197.11.26.213  |Ciano       |Atleta De Arremesso De Peso|(031) 0803-6753    |
|398.471.625-73|felipepires@uol.com.br            |*Aw5EOAvy9|1921-11-11        |MG    |2023-01-02  

In [6]:
df.printSchema()

root
 |-- cpf: string (nullable = true)
 |-- email: string (nullable = true)
 |-- senha: string (nullable = true)
 |-- data_de_nascimento: date (nullable = true)
 |-- estado: string (nullable = true)
 |-- data_cadastro: date (nullable = true)
 |-- ipv4: string (nullable = true)
 |-- cor_favorita: string (nullable = true)
 |-- profissao: string (nullable = true)
 |-- telefone: string (nullable = true)



In [26]:
(
    df
    # funçao que substitui valores de uma coluna especifca, neste exemplo na coluna telefone substitui 'espaco' por 'nada'
    # cast transforma para numerico
    .withColumn('tel', F.regexp_replace('telefone', ' ', '').cast('int'))
    
    # neste caso o regexp_replace mantem somente os numeros
    # .withColumn('tel', F.regexp_replace('telefone', '(\D)', ''))
    
    # ordernar 
    # .orderBy(F.asc('tel'))
    # .orderBy(F.desc('tel'))
    
    # ordernar deixando os nulos para comeco, pode ser usado desc
    # .orderBy(F.asc_nulls_first('tel'))
    
    # ordernar deixando os nulos para final, pode ser usado desc
    # .orderBy(F.asc_nulls_last('tel'))
    
    # funcao between (intervalo)
    # .where(F.year('data_de_nascimento').between(2010,2015))
    
    # funcao de filtro que contem a palavra joao
    # .where(F.col('email').contains('joao'))
    
    # todas as palavras que terminam com uma palavra especifica
    # .where(F.col('email').endswith('.br'))
    
    # todas as palavras que comecam com uma palavra especifica
    # .where(F.col('email').startswith('d'))
    
    # neste caso traz todo mundo que é de SP e RJ
    # .where(F.col('estado').isin('RJ', 'SP'))
    
    # apenas nulos
    # .where(F.col('tel').isNull())
    
    # todos que nao sao nulos
    # .where(F.col('tel').isNotNull())
    # outra forma de negar uma condição
    .where(~F.col('tel').isNull())
    
    
    .show(20, False)
)

+--------------+------------------------------+----------+------------------+------+-------------+---------------+------------+---------------------------------+------------+----------+
|cpf           |email                         |senha     |data_de_nascimento|estado|data_cadastro|ipv4           |cor_favorita|profissao                        |telefone    |tel       |
+--------------+------------------------------+----------+------------------+------+-------------+---------------+------------+---------------------------------+------------+----------+
|217.639.540-99|jribeiro@bol.com.br           |MEf1X7fj_0|2021-10-05        |PA    |2023-03-02   |71.22.224.5    |Marrom      |Geólogo                          |21 1432 4092|2114324092|
|136.894.750-66|barrossarah@bol.com.br        |zm1eLH9t%o|1975-05-21        |CE    |2023-01-11   |79.204.112.52  |Magenta     |Auxiliar De Serviços Gerais      |11 7350 7069|1173507069|
|748.591.320-41|nalves@yahoo.com.br           |$U0HLVZti5|1931-12-14  